In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

In [5]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [7]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [8]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [9]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informação geral e detalhada no google sobre um tópico.",
        tools=[google_search],
        instruction="""
       Função: Agente especializado em buscar e coletar dados e informações
       relevantes em (Google Search).
       Fornecer pesquisa detalhadas e relevantes para o planejamento e organização de conteúdo.
       Oferecer apenas as pesquisas mais relevantes.
       **Instruções de Busca Específicas:**
       1. **Foco Principal:** Busque por **informações aprofundadas, dados, fatos, tendências, contextos e detalhes** sobre o tópico fornecido. O objetivo é obter um entendimento completo e factual do assunto.
       2. **Conteúdo a Excluir Ativamente:**
          - Cursos online, treinamentos, webinars, plataformas de ensino à distância, materiais de curso.
       3. **Recomendações (Livros, etc.):** Se encontrar recomendações relevantes (como livros) que **complementam ou ilustram a informação geral** sobre o tópico, pode incluí-las. No entanto, o foco **não** é criar uma lista de recomendações, mas sim usar essas recomendações como exemplos ou fontes dentro de um contexto informativo mais amplo.

       Identificar e extrair informações chave, dados, tendências e exemplos **que expliquem e detalhem o tópico**.
       Summarizar os achados de forma clara e concisa, priorizando a apresentação das **informações informativas e factuais** encontradas. Mencione recomendações apenas se forem altamente relevantes para o conteúdo informativo.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    resposta_do_agente_buscador = call_agent(buscador, entrada_do_agente_buscador)
    return resposta_do_agente_buscador

In [10]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Função: Agente especializado em estruturar e organizar informações para
        criar planos de conteúdo detalhados e acionáveis para a redação final.
        Baseado no Tópico e nos dados de Lançamentos buscados, crie um plano
        de conteúdo organizado em SEÇÕES ENUMERADAS em Markdown.

        Exemplo de formato desejado:
        1. Título da Primeira Seção (ex: tema 1)
        2. Título da Segunda Seção (ex: tema 2)
        3. Título da Terceira Seção (ex: tema 3)
        4. Titulo da Quarta Seção (ex: tema 4)

        Certifique-se de que cada ponto numerado é um título de seção claro e conciso.
        Filtrar os dados recebidos pelo agente_buscador selecionando apenas 4 mais relevantes e importantes
        abordar.
        NÃO GERAR MAIS DE 4 SEÇÕES.
        Selecionar sitações, pesquisas ou fatos interessantes para abordar nos paragrafos.
        O plano deve ser uma lista numerada que delineia a estrutura completa do post.
        """,
        description="Planejador de conteúdo",
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [11]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
        Você é um redator Criativo especializado em linguagem e gramatica,
        com foco em criar redações e resumos tecnicos profissionais.
        Você escreve para alunos e profissionais buscando aprimorar seus conhecimentos.
        Escrita profissional mas com toque didático.
        Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
        escreva o texto abordando o tema.
        Importante ressaltar para deixar pelo menos 1 citação relevante.
        O post deve ser engajador, informativo, que prenda a tenção e foco do leitor.
        O texto deve seguir um padrão de no máximo 300 palavras.
            """,
        description="Agente redator de posts engajador"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [14]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um revisor Criativo especializado em linguagem e gramatica,
        com foco em criar redações e resumos tecnicos.
        Você escreve para alunos e profissionais buscando aprimorar seus conhecimentos.
        Escrita profissional mas com toque didático. Seja conciso em suas redações.
        Revise o rascunho de post abaixo sobre o tópico indicado, verificando
        sua escrita, **assegurando que a Forma de Abordagem e o Propósito originais sejam mantidos**.
        Não desejo que seja exigente e nem faça muitas correções
        Apenas exija mudança se for realmente importante e necessário para
        melhoria do material original.
        Se o rascunho estiver bom, responda apenas 'O rascunho está pronto!'.
        O texto deve seguir um padrão de no máximo 300 palavras.
        NÃO FAZER CORREÇÃO após 2 tentativas.
        """,
        description="Agente revisor de post."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [17]:
from datetime import date
import re
from IPython.display import display, Markdown
from google.adk.agents import Agent  # Import Agent
from google.adk.runners import Runner # Import Runner
from google.adk.sessions import InMemorySessionService # Import InMemorySessionService
from google.adk.tools import google_search # Import google_search
from google.genai import types  # Para criar conteúdos (Content e Part) # Import types
import textwrap
import os
from google.colab import userdata

# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

def extract_content_from_part(part):
    """Extracts text content from a GenAI Part."""
    if part.text:
        return part.text
    return ""

def call_agent(agent: Agent, message_text: str) -> str:
    """Calls an agent with a message and returns the final response."""
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    try:
        for event in runner.run(user_id="user1", session_id="session1", new_message=content):
            if event.is_final_response():
              for part in event.content.parts:
                final_response += extract_content_from_part(part)
                final_response += "\n"
    except Exception as e:
        print(f"Error during agent run: {e}")
        # Retornar uma mensagem de erro ou rascunho parcial em caso de falha
        return f"Erro ao chamar o agente {agent.name}: {e}"
    return final_response

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def extrair_secoes_do_plano(texto_plano: str) -> list[str]:
    secoes = []
    # Padrões mais robustos para extrair títulos de seção
    padroes_secao = [
        r'^\s*##\s*(.+)',  # Títulos em Markdown ##
        r'^\s*\d+\.\s*(.+)', # Listas numeradas 1.
        r'^\s*-\s*(.+)'    # Listas com traço -
    ]
    linhas = texto_plano.splitlines()
    for linha in linhas:
        for padrao in padroes_secao:
            match = re.match(padrao, linha.strip())
            if match:
                secao_titulo = match.group(1).strip()
                # Adicionar uma verificação para evitar seções vazias ou muito curtas
                if secao_titulo and len(secao_titulo) > 3:
                    secoes.append(secao_titulo)
                break # Parar após encontrar o primeiro padrão correspondente na linha
    # Adicionar uma seção padrão se nenhuma for encontrada, mas houver algum texto
    if not secoes and texto_plano.strip():
         print("Aviso: Nenhuma seção clara encontrada no plano. Usando 'Conteúdo Principal'.")
         return ["Conteúdo Principal"]
    elif not secoes and not texto_plano.strip():
        print("Aviso: O plano de conteúdo está vazio.")
        return []
    return secoes


# --- Definição dos Agentes ---
buscador_agent_obj = Agent(
    name="agente_buscador",
    model="gemini-2.0-flash",
    description="Agente que busca informação geral e detalhada no google sobre um tópico.",
    tools=[google_search],
    instruction="""
   Função: Agente especializado em buscar e coletar dados e informações
   relevantes em (Google Search).
   Fornecer pesquisa detalhadas e relevantes para o planejamento e organização de conteúdo.
   Oferecer apenas as pesquisas mais relevantes.
   **Instruções de Busca Específicas:**
   1. **Foco Principal:** Busque por **informações aprofundadas, dados, fatos, tendências, contextos e detalhes** sobre o tópico fornecido, **considerando o foco e propósito especificados pelo usuário**. O objetivo é obter um entendimento completo e factual do assunto, **alinhado com a forma de abordagem desejada**.
   2. **Conteúdo a Excluir Ativamente:**
      - Cursos online, treinamentos, webinars, plataformas de ensino à distância, materiais de curso.
   3. **Recomendações (Livros, etc.):** Se encontrar recomendações relevantes (como livros) que **complementam ou ilustram a informação geral** sobre o tópico, pode incluí-las. No entanto, o foco **não** é criar uma lista de recomendações, mas sim usar essas recomendações como exemplos ou fontes dentro de um contexto informativo mais amplo.

   Identificar e extrair informações chave, dados, tendências e exemplos **que expliquem e detalhem o tópico**, **seguindo a forma de abordagem e o propósito definidos pelo usuário**.
   Summarizar os achados de forma clara e concisa, priorizando a apresentação das **informações informativas e factuais** encontradas. Mencione recomendações apenas se forem altamente relevantes para o conteúdo informativo.
    """
)

planejador_agent_obj = Agent(
    name="agente_planejador",
    model="gemini-2.0-flash",
    instruction="""
    Função: Agente especializado em estruturar e organizar informações para
    criar planos de conteúdo detalhados e acionáveis para a redação final.
    Baseado no Tópico, Forma de Abordagem, Propósito e nos dados de Lançamentos buscados, crie um plano
    de conteúdo organizado em SEÇÕES ENUMERADAS em Markdown.

    Exemplo de formato desejado:
    1. Título da Primeira Seção (ex: tema 1)
    2. Título da Segunda Seção (ex: tema 2)
    3. Título da Terceira Seção (ex: tema 3)
    4. Titulo da Quarta Seção (ex: tema 4)

    Certifique-se de que cada ponto numerado é um título de seção claro e conciso.
    Filtrar os dados recebidos pelo agente_buscador selecionando apenas 4 mais relevantes e importantes
    abordar, **considerando a Forma de Abordagem e o Propósito especificados**.
    NÃO GERAR MAIS DE 4 SEÇÕES.
    Selecionar sitações, pesquisas ou fatos interessantes para abordar nos paragrafos, **alinhados com a Forma de Abordagem e Propósito**.
    O plano deve ser uma lista numerada que delineia a estrutura completa do post.
    """,
    description="Planejador de conteúdo",
)

redator_agent_obj = Agent(
    name="agente_redator",
    model="gemini-2.0-flash",
    instruction="""
        Você é um redator Criativo especializado em linguagem e gramatica,
        com foco em criar redações e resumos tecnicos profissionais.
        Você escreve para alunos e profissionais buscando aprimorar seus conhecimentos.
        Escrita profissional mas com toque didático.
        Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
        escreva o texto abordando o tema, **mantendo a Forma de Abordagem e o Propósito originais**.
        Importante ressaltar para deixar pelo menos 1 citação relevante.
        O post deve ser engajador, informativo, que prenda a tenção e foco do leitor.
        O texto deve seguir um padrão de no máximo 300 palavras.

        """,
    description="Agente redator de posts engajador"
)

revisor_agent_obj = Agent(
    name="agente_revisor",
    model="gemini-2.0-flash",
    instruction="""
        Você é um revisor Criativo especializado em linguagem e gramatica,
        com foco em criar redações e resumos tecnicos.
        Você escreve para alunos e profissionais buscando aprimorar seus conhecimentos.
        Escrita profissional mas com toque didático. Seja conciso em suas redações.
        Revise o rascunho de post abaixo sobre o tópico indicado, verificando
        sua escrita, **assegurando que a Forma de Abordagem e o Propósito originais sejam mantidos**.
        Não desejo que seja exigente e nem faça muitas correções
        Apenas exija mudança se for realmente importante e necessário para
        melhoria do material original.
        Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto!'.
        O texto deve seguir um padrão de no máximo 300 palavras.
        NÃO FAZER CORREÇÃO após 2 tentativas.
        """,
    description="Agente revisor de post."
)


# --- Lógica Principal de Orquestração do Chatbot ---

## 🚀 Iniciando o Sistema de Planejamento e Geração de Conteúdo 🚀

data_de_hoje = date.today().strftime("%d/%m/%Y")

### ❓ Obtendo o Tópico e Preferências do Usuário

print("## 📚 Inicie o Planejamento do seu Conteúdo! 📚")

print("\nPor favor, digite o TÓPICO principal sobre o qual você quer estudar/criar conteúdo:")
topico = input("Tópico: ")

if not topico:
  print("❌ Nenhum tópico inserido. Por favor, insira um tópico válido para continuar.")
else:
  print(f"\n✅ Tópico recebido: {topico}.")

  print("\nCom qual FORMA DE ABORDAGEM você deseja que o tema seja tratado? (Ex: Técnica, Didática, Histórica, Comparativa, etc.)")
  forma_abordagem = input("Forma de Abordagem: ")

  print("\nQual o PROPÓSITO principal deste conteúdo? (Ex: Explicar um conceito, Comparar tecnologias, Apresentar tendências, Guia prático, etc.)")
  proposito_conteudo = input("Propósito do Conteúdo: ")

  print(f"\n✅ Preferências capturadas:")
  print(f"- Tópico: {topico}")
  print(f"- Forma de Abordagem: {forma_abordagem if forma_abordagem else 'Não especificada'}")
  print(f"- Propósito: {proposito_conteudo if proposito_conteudo else 'Não especificado'}")

  print("\nIniciando o processo de busca e planejamento...")

  ## 🔍 Etapa 1: Buscando Informações Relevantes (Agente Buscador)

  print("\n--- Executando Agente Buscador ---")
  # Chama o agente buscador para coletar dados sobre o tópico
  # Inclua as preferências do usuário na entrada do agente buscador
  entrada_do_agente_buscador = f"""
  Tópico: {topico}
  Data de hoje: {data_de_hoje}
  Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
  Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}
  """
  lançamentos_buscados = call_agent(buscador_agent_obj, entrada_do_agente_buscador)

  print("\n### Resultado da Busca:")
  # Exibe os resultados da busca formatados em Markdown
  display(to_markdown(lançamentos_buscados))
  print("-------------------------------")

  ## 📝 Etapa 2: Planejando a Estrutura do Conteúdo (Agente Planejador)

  print("\n--- Executando Agente Planejador ---")
  # Chama o agente planejador para criar um plano de conteúdo baseado na busca
  # Inclua as preferências do usuário na entrada do agente planejador
  entrada_do_agente_planejador = f"""
  Tópico: {topico}
  Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
  Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}
  Lançamentos buscados: {lançamentos_buscados}
  """
  plano_de_post = call_agent(planejador_agent_obj, entrada_do_agente_planejador)

  print("\n### Plano de Conteúdo Gerado:")
  # Exibe o plano de conteúdo formatado em Markdown
  display(to_markdown(plano_de_post))
  print("-------------------------------")

  ## ✍️ Etapa 3: Gerando e Revisando Conteúdo por Seção (Agente Redator & Revisor)

  print("\n--- Iniciando Geração e Revisão do Rascunho (por Seção) ---")

  # Extrai as seções do plano de conteúdo para processamento individual
  secoes_do_plano = extrair_secoes_do_plano(plano_de_post)

  # Crie uma lista para armazenar os rascunhos de cada seção
  rascunhos_individuais_secoes = []

  if not secoes_do_plano:
      print("⚠️ Aviso: Não foi possível extrair seções do plano. Não será possível gerar o rascunho por seção.")
      rascunho_do_post = "" # Define rascunho_do_post como vazio se não houver seções
  else:
      rascunho_do_post_partes = [] # Lista para armazenar as partes do rascunho por seção
      MAX_TENTATIVAS_REVISAO = 3 # Limite para o ciclo de revisão por seção

      # Loop sobre cada seção do plano
      for i, secao in enumerate(secoes_do_plano):
          print(f"\n--- Processando Seção {i+1}/{len(secoes_do_plano)}: '{secao}' ---")

          ### ✏️ Gerando Rascunho Inicial da Seção

          # Inclua as preferências do usuário na entrada para o redator
          entrada_para_redator_secao = f"""
          Tópico Geral: {topico}
          Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
          Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}

          Plano de Conteúdo Completo:
          {plano_de_post}

          ---
          POR FAVOR, ESCREVA APENAS O CONTEÚDO DA SEÇÃO ESPECÍFICA ABAIXO.
          Não inclua o título da seção no início da sua resposta.
          Seção a escrever: {secao}

          Instruções Adicionais para esta Seção:
          (Se houver instruções específicas para esta seção do plano, adicione aqui)
          """
          rascunho_da_secao = call_agent(redator_agent_obj, entrada_para_redator_secao)
          print("\n#### Rascunho Inicial:")
          display(to_markdown(rascunho_da_secao))
          print("-------------------------------")

          ### 👀 Ciclo de Revisão da Seção

          revisao_finalizada = False
          tentativas_revisao = 0

          # Loop de revisão até aprovação ou limite de tentativas
          while not revisao_finalizada and tentativas_revisao < MAX_TENTATIVAS_REVISAO:
              tentativas_revisao += 1
              print(f"\n--- Revisando Seção (Tentativa {tentativas_revisao}/{MAX_TENTATIVAS_REVISAO}) ---")
              # --- Adicionados logs de depuração ---
              print(f"Estado inicial da revisão: revisao_finalizada={revisao_finalizada}, tentativas_revisao={tentativas_revisao}")
              # --- Fim dos logs de depuração ---


              entrada_para_revisor_secao = f"""
              Tópico Geral: {topico}
              Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
              Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}
              Plano de Conteúdo Relevante para esta Seção: {plano_de_post}

              ---
              Rascunho da Seção "{secao}" para Revisão:
              {rascunho_da_secao}

              Por favor, revise o rascunho da seção acima. Se estiver bom, responda APENAS a frase exata 'O rascunho está ótimo e pronto!'. Caso contrário, aponte os problemas e sugira melhorias específicas para que o redator possa corrigir. Certifique-se de que a revisão leva em conta a Forma de Abordagem e o Propósito originais.
              """

              feedback_revisor = call_agent(revisor_agent_obj, entrada_para_revisor_secao)
              print("\n#### Feedback do Revisor:")
              display(to_markdown(feedback_revisor))
              print("-------------------------------")

              # --- Adicionados logs de depuração ---
              print(f"Feedback bruto do revisor: '{feedback_revisor}'")
              print(f"Feedback processado (minusculo/sem espaços): '{feedback_revisor.strip().lower()}'")
              # --- Fim dos logs de depuração ---

              if feedback_revisor.strip().lower() == 'o rascunho está ótimo e pronto!':
                  print(f"👍 Seção '{secao}' aprovada pelo Revisor!")
                  revisao_finalizada = True
                  # --- Adicionados logs de depuração ---
                  print(f"Valor de revisao_finalizada após aprovação: {revisao_finalizada}")
                  # --- Fim dos logs de depuração ---
              else:
                  print(f"🔄 Seção '{secao}' precisa de correções. Enviando feedback para o Redator.")
                  entrada_para_redator_correcao = f"""
                  Tópico Geral: {topico}
                  Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
                  Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}
                  Plano de Conteúdo Completo:
                  {plano_de_post}
                  ---
                  POR FAVOR, REESCREVA/CORRIJA APENAS O CONTEÚDO DA SEÇÃO ESPECÍFICA ABAIXO, BASEADO NO FEEDBACK.
                  Não inclua o título da seção no início da sua resposta.
                  Seção a corrigir: {secao}
                  Rascunho Anterior Desta Seção:
                  {rascunho_da_secao}
                  ---
                  Feedback do Revisor para Correção:
                  {feedback_revisor}
                  Por favor, incorpore as sugestões de correção do revisor no novo rascunho desta seção, garantindo que a Forma de Abordagem e o Propósito sejam mantidos.
                  """
                  rascunho_da_secao = call_agent(redator_agent_obj, entrada_para_redator_correcao)
                  print("\n#### Rascunho Corrigido:")
                  display(to_markdown(rascunho_da_secao))
                  print("-------------------------------")

              # --- Adicionados logs de depuração ---
              print(f"Fim da Tentativa {tentativas_revisao}. Condição do loop: {not revisao_finalizada and tentativas_revisao < MAX_TENTATIVAS_REVISAO}")
              # --- Fim dos logs de depuração ---


          if not revisao_finalizada:
              print(f"⚠️ Aviso: A Seção '{secao}' não foi aprovada após {MAX_TENTATIVAS_REVISAO} tentativas. Usando a última versão gerada.")

          # rascunhos_individuais_secoes.append(f"## {secao}\n\n{rascunho_da_secao}") # Remova esta linha
          rascunho_do_post_partes.append(rascunho_da_secao) # Adiciona apenas o conteúdo do rascunho

      # Reconstroi o rascunho final adicionando os títulos de seção antes de cada parte do conteúdo
      rascunho_do_post_final_formatado = []
      for i, secao in enumerate(secoes_do_plano):
          # Adiciona o título da seção e o conteúdo correspondente
          rascunho_do_post_final_formatado.append(f"## {secao}\n\n{rascunho_do_post_partes[i]}")

      rascunho_do_post = "\n\n".join(rascunho_do_post_final_formatado)


      print("\n--- Geração e Revisão por Seção Concluídas ---")
      print("\n## 🎉 Rascunho Completo Gerado (Após Revisão por Seção) 🎉")
      display(to_markdown(rascunho_do_post))
      print("-------------------------------")


  print("\nO sistema concluiu o processo de planejamento e geração de conteúdo.")

## 📚 Inicie o Planejamento do seu Conteúdo! 📚

Por favor, digite o TÓPICO principal sobre o qual você quer estudar/criar conteúdo:
Tópico: Inteligencia artificial

✅ Tópico recebido: Inteligencia artificial.

Com qual FORMA DE ABORDAGEM você deseja que o tema seja tratado? (Ex: Técnica, Didática, Histórica, Comparativa, etc.)
Forma de Abordagem: didática

Qual o PROPÓSITO principal deste conteúdo? (Ex: Explicar um conceito, Comparar tecnologias, Apresentar tendências, Guia prático, etc.)
Propósito do Conteúdo: apresentar tendencias

✅ Preferências capturadas:
- Tópico: Inteligencia artificial
- Forma de Abordagem: didática
- Propósito: apresentar tendencias

Iniciando o processo de busca e planejamento...

--- Executando Agente Buscador ---

### Resultado da Busca:


> Para apresentar as tendências da inteligência artificial de forma didática, vou realizar algumas pesquisas para coletar informações relevantes e atuais.
> 
> 
> Com base nas pesquisas realizadas, apresento as tendências da inteligência artificial para 2025 de forma didática:
> 
> 1.  **Agentes de IA e Autonomia:**
> 
>     *   Os agentes de IA estão se tornando mais autônomos, capazes de gerenciar tarefas complexas com pouca intervenção humana. Eles podem tomar decisões com base em dados e algoritmos, otimizando processos e liberando equipes para atividades estratégicas.
>     *   Exemplo: Agentes de IA preenchendo automaticamente formulários com base em informações coletadas de e-mails, cabendo aos humanos supervisionar os resultados.
> 
> 2.  **IA Generativa como Padrão:**
> 
>     *   Ferramentas como ChatGPT e MidJourney estão se expandindo e se tornando essenciais na criação de conteúdo, marketing, design e estratégias de vendas. A IA generativa acelera a produção de materiais e automatiza procedimentos demorados.
>     *   Exemplo: 80% dos usuários do ChatGPT relatam um aumento significativo na produtividade ao criar textos, imagens e vídeos.
> 
> 3.  **Hiperpersonalização no Atendimento ao Cliente:**
> 
>     *   A IA permite a criação de experiências únicas e personalizadas para cada cliente, ajustando propostas e ofertas com base em suas necessidades individuais.
>     *   Exemplo: 63% dos consumidores preferem comprar de empresas que oferecem experiências personalizadas.
> 
> 4.  **Democratização da IA:**
> 
>     *   Ferramentas no-code e low-code tornam a IA mais acessível para empresas de todos os tamanhos, eliminando a necessidade de conhecimento técnico avançado. Isso permite que equipes criem soluções personalizadas, como automação de fluxos e análises preditivas.
>     *   Exemplo: Até 2026, mais de 80% dos fornecedores de software integrarão IA generativa em seus produtos, e 70% das novas aplicações serão desenvolvidas com ferramentas no-code ou low-code.
> 
> 5.  **IA Multimodal:**
> 
>     *   A IA multimodal processa diferentes tipos de dados simultaneamente, como texto, imagem, áudio e vídeo, permitindo uma compreensão mais completa do contexto.
>     *   Exemplo: A IA analisa posts nas redes sociais, interpretando o texto e as imagens associadas para captar nuances como ironia ou emoção.
> 
> 6.  **Colaboração entre Pessoas e IA:**
> 
>     *   A IA assume o papel de assistente dos profissionais, auxiliando na execução de tarefas e permitindo que os humanos se concentrem em atividades estratégicas e criativas.
>     *   Exemplo: Agentes de IA auxiliando profissionais a gerenciar seu tempo e criar espaço para o pensamento estratégico.
> 
> 7.  **IA e o Futuro do Trabalho:**
> 
>     *   A IA está remodelando o mercado de trabalho, criando novas oportunidades e exigindo novas habilidades. Profissões ligadas à análise de dados, desenvolvimento de IA, segurança cibernética e experiência do cliente devem crescer.
>     *   Exemplo: A IA deve criar cerca de 97 milhões de novos empregos até 2025, ao mesmo tempo em que elimina aproximadamente 85 milhões de postos de trabalho tradicionais.
> 
> 8.  **Tendências Adicionais:**
> 
>     *   **Computação Quântica:** A computação quântica promete acelerar a inovação em IA e aprimorar suas capacidades, permitindo o processamento de grandes volumes de dados mais rapidamente.
>     *   **Personalização com IA:** A IA permite a criação de propostas altamente personalizadas, ajustadas às necessidades de cada cliente.
>     *   **Ética e Responsabilidade:** Há um crescente foco em garantir que a IA seja usada de forma ética e responsável, abordando questões como viés algorítmico e privacidade de dados.
>     *   **Integração com IoT e Edge Computing:** A IA está sendo cada vez mais integrada com dispositivos IoT e edge computing, permitindo que dispositivos inteligentes tomem decisões mais rápidas e eficientes.
>     *   **Segurança:** A segurança das soluções de IA é uma prioridade máxima, com foco no investimento em segurança para proteger os sistemas contra possíveis ataques.
> 
> Em resumo, a inteligência artificial está evoluindo rapidamente e se tornando uma parte essencial de diversos setores. As tendências para 2025 apontam para uma IA mais autônoma, acessível, personalizada e colaborativa, com um impacto significativo no mercado de trabalho e na forma como interagimos com a tecnologia.
> 


-------------------------------

--- Executando Agente Planejador ---

### Plano de Conteúdo Gerado:


> 1.  **IA Generativa e a Expansão da Criação de Conteúdo:**
>     *   Como a IA generativa, exemplificada por ferramentas como ChatGPT e MidJourney, está se tornando padrão na criação de conteúdo e em outras áreas.
>     *   Impacto da IA generativa na produtividade e automação de tarefas demoradas, com dados estatísticos sobre o aumento da eficiência.
> 
> 2.  **Hiperpersonalização Impulsionada por IA no Atendimento ao Cliente:**
>     *   A capacidade da IA de criar experiências personalizadas para clientes, ajustando ofertas e propostas com base em necessidades individuais.
>     *   Estatísticas que demonstram a preferência dos consumidores por empresas que oferecem experiências personalizadas.
> 
> 3.  **Democratização da IA Através de Ferramentas No-Code e Low-Code:**
>     *   Como ferramentas no-code e low-code estão tornando a IA mais acessível para empresas de todos os tamanhos, eliminando a necessidade de conhecimento técnico avançado.
>     *   Projeções sobre a integração da IA generativa em softwares e o desenvolvimento de novas aplicações com ferramentas no-code ou low-code.
> 
> 4.  **IA Multimodal e a Compreensão Contextual Avançada:**
>     *   A capacidade da IA multimodal de processar simultaneamente diferentes tipos de dados, como texto, imagem, áudio e vídeo, para uma compreensão mais completa do contexto.
>     *   Exemplos práticos de como a IA multimodal é utilizada para interpretar nuances e emoções em posts nas redes sociais, analisando tanto o texto quanto as imagens associadas.
> 


-------------------------------

--- Iniciando Geração e Revisão do Rascunho (por Seção) ---

--- Processando Seção 1/4: '**IA Generativa e a Expansão da Criação de Conteúdo:**' ---

#### Rascunho Inicial:


...

-------------------------------

--- Revisando Seção (Tentativa 1/3) ---
Estado inicial da revisão: revisao_finalizada=False, tentativas_revisao=1

#### Feedback do Revisor:


> O rascunho está ótimo e pronto!


-------------------------------
Feedback bruto do revisor: 'O rascunho está ótimo e pronto!
'
Feedback processado (minusculo/sem espaços): 'o rascunho está ótimo e pronto!'
👍 Seção '**IA Multimodal e a Compreensão Contextual Avançada:**' aprovada pelo Revisor!
Valor de revisao_finalizada após aprovação: True
Fim da Tentativa 1. Condição do loop: False

--- Geração e Revisão por Seção Concluídas ---

## 🎉 Rascunho Completo Gerado (Após Revisão por Seção) 🎉


> ## **IA Generativa e a Expansão da Criação de Conteúdo:**
> 
> A Inteligência Artificial (IA) generativa está revolucionando a criação de conteúdo. Ferramentas como ChatGPT e MidJourney tornaram-se padrão, impulsionando a produtividade e automatizando tarefas antes demoradas.
> 
> Imagine aumentar a eficiência da sua equipe em até 50% com a ajuda da IA generativa. Essa tecnologia não só economiza tempo, mas também abre portas para a inovação, permitindo que os profissionais se concentrem em tarefas estratégicas e criativas.
> 
> "A inteligência artificial é a nova eletricidade." – Andrew Ng.
> 
> Essa expansão da IA generativa democratiza a criação de conteúdo, capacitando empresas e criadores a produzirem materiais de alta qualidade de forma mais rápida e acessível.
> 
> 
> ## **Hiperpersonalização Impulsionada por IA no Atendimento ao Cliente:**
> 
> ## Hiperpersonalização Impulsionada por IA no Atendimento ao Cliente
> 
> Já imaginou um mundo onde cada interação com uma empresa é única, feita sob medida para você? A Inteligência Artificial está tornando isso realidade! Com a capacidade de analisar dados e entender as necessidades individuais de cada cliente, a IA permite criar experiências hiperpersonalizadas.
> 
> Imagine receber ofertas de produtos que você realmente quer, ou ter um atendimento que já sabe do seu histórico e preferências. A IA possibilita ajustar ofertas e propostas com base em suas necessidades, transformando a forma como as empresas se relacionam com seus clientes.
> 
> Não é à toa que os consumidores estão cada vez mais exigentes e preferem empresas que oferecem experiências personalizadas. "A personalização não é mais uma opção, mas uma expectativa do cliente", como disse Tom Fishburne, renomado especialista em marketing. Invista em IA e destaque-se!
> 
> 
> 
> ## **Democratização da IA Através de Ferramentas No-Code e Low-Code:**
> 
> A Inteligência Artificial está se tornando cada vez mais acessível, e as ferramentas No-Code e Low-Code são as grandes responsáveis por essa democratização. Elas permitem que empresas de todos os portes implementem soluções de IA sem a necessidade de especialistas em programação, abrindo um leque de oportunidades para inovação e crescimento.
> 
> Imagine poder criar um chatbot inteligente para o seu negócio sem escrever uma única linha de código, ou então, automatizar tarefas complexas com apenas alguns cliques. Essa é a realidade que as plataformas No-Code e Low-Code proporcionam, simplificando o desenvolvimento e a implementação de soluções baseadas em IA.
> 
> A integração da IA generativa em softwares e o desenvolvimento de novas aplicações com essas ferramentas é uma tendência que veio para ficar. Em breve, veremos um boom de soluções inovadoras criadas por pessoas que antes não tinham acesso ao mundo da Inteligência Artificial.
> 
> Como disse Andrew Ng, um dos maiores nomes da área, "IA é a nova eletricidade". E assim como a eletricidade transformou o mundo, a IA está revolucionando a forma como vivemos e trabalhamos. Com a democratização da IA, o futuro está ao alcance de todos.
> 
> 
> 
> ## **IA Multimodal e a Compreensão Contextual Avançada:**
> 
> A Inteligência Artificial multimodal chegou para revolucionar a forma como as máquinas "entendem" o mundo. Imagine um sistema que não apenas lê um texto, mas também observa imagens, escuta áudios e assiste a vídeos simultaneamente. Essa é a proposta da IA multimodal: uma compreensão contextual muito mais rica e profunda.
> 
> Na prática, isso significa que a IA pode, por exemplo, analisar um post nas redes sociais e identificar nuances e emoções que passariam despercebidas se ela avaliasse apenas o texto. Ao combinar a análise do texto com as imagens e os vídeos associados, a IA multimodal consegue interpretar o contexto de forma muito mais precisa.
> 
> "A inteligência é a habilidade de se adaptar à mudança", já dizia Stephen Hawking. E a IA multimodal representa exatamente isso: a capacidade da tecnologia de se adaptar e evoluir para compreender o mundo de forma cada vez mais completa e contextualizada.
> 


-------------------------------

O sistema concluiu o processo de planejamento e geração de conteúdo.
